In [ ]:
from collections import deque
import time
import math
from datetime import datetime, timedelta
import pandas as pd

In [7]:
from datetime import datetime, timedelta

class TimeSlots:
    def __init__(self, start_time="10:00", end_time="23:55", interval=5):
        self.start_time = datetime.strptime(start_time, "%H:%M")
        self.end_time = datetime.strptime(end_time, "%H:%M")
        self.interval = timedelta(minutes=interval)
        self.slots = self.generate_time_slots()
    
    def generate_time_slots(self):
        slots = []
        current_time = self.start_time
        while current_time <= self.end_time:
            slots.append(current_time.strftime("%H:%M"))
            current_time += self.interval
        return slots
    
    def display_slots(self):
        for slot in self.slots:
            print(slot)

# Create an instance of TimeSlots
time_slots = TimeSlots()

# Display the time slots
time_slots.display_slots()

# This fuction create a list of time instances 


10:00
10:05
10:10
10:15
10:20
10:25
10:30
10:35
10:40
10:45
10:50
10:55
11:00
11:05
11:10
11:15
11:20
11:25
11:30
11:35
11:40
11:45
11:50
11:55
12:00
12:05
12:10
12:15
12:20
12:25
12:30
12:35
12:40
12:45
12:50
12:55
13:00
13:05
13:10
13:15
13:20
13:25
13:30
13:35
13:40
13:45
13:50
13:55
14:00
14:05
14:10
14:15
14:20
14:25
14:30
14:35
14:40
14:45
14:50
14:55
15:00
15:05
15:10
15:15
15:20
15:25
15:30
15:35
15:40
15:45
15:50
15:55
16:00
16:05
16:10
16:15
16:20
16:25
16:30
16:35
16:40
16:45
16:50
16:55
17:00
17:05
17:10
17:15
17:20
17:25
17:30
17:35
17:40
17:45
17:50
17:55
18:00
18:05
18:10
18:15
18:20
18:25
18:30
18:35
18:40
18:45
18:50
18:55
19:00
19:05
19:10
19:15
19:20
19:25
19:30
19:35
19:40
19:45
19:50
19:55
20:00
20:05
20:10
20:15
20:20
20:25
20:30
20:35
20:40
20:45
20:50
20:55
21:00
21:05
21:10
21:15
21:20
21:25
21:30
21:35
21:40
21:45
21:50
21:55
22:00
22:05
22:10
22:15
22:20
22:25
22:30
22:35
22:40
22:45
22:50
22:55
23:00
23:05
23:10
23:15
23:20
23:25
23:30
23:35
23:40
23:45
23:5

In [ ]:
class Visitor: # generate unique Visitor
    def __init__(self, id, itinerary, fast_pass: int, arrival_time):
        """
        :param id: a unique ID
        :param itinerary: a queue of itinerary (where they will go, in sequence)
        :param fast_pass: 0 for regular ticket, 1 for fast-pass ticket
        """
        self.id = id
        self.fast_pass = fast_pass 
        self.itinerary = itinerary 
        self.current_time = arrival_time # First set to when they are 'spawned'
        self.accumulated_waiting_time = 0 ### LINK TO JON's part
        self.accumulated_satisfactory_score = 0 ### LINK TO JON's part
        
        self.current_action = None
        self.current_ride_start = None # The moment they get served from the queue
        
        self.service_start_time = None
        self.service_end_time = None

    def __repr__(self): # representation method 
        return f"Visitor {self.id, self.fast_pass}"
    
    def choose_action(self):
        self.current_action = self.itinerary.popleft()
        return self.current_action
    
    def travel(self, edge): # visitor travels between nodes ### link to Jon's
        """
        :param edge: the edge from graph.py
        """
        time_travelled = edge.distance # It takes 1 minute to travel distance = 1
        self.current_time += timedelta(minutes=time_travelled)



In [ ]:
class VisitorList: # List of visitor for a single attraction/utility throughout the whole day
    def __init__(self, name):
        """
        :param name: The name of the attraction/utility
        """
        self.name = name
        self.name_list = []

    def __repr__(self): # representation method 
        return f"Visitor List for {self.name}"
    
    def add_visitor_to_list(self, visitor):
        """
        :param visitor: a Visitor object
        """
        if isinstance(visitor, Visitor):
            self.name_list.append(visitor)

In [ ]:
class Attraction: # Including shows, rides
    def __init__(self, name: str, ride_duration: int, ride_capacity: int):
        """
        Initialize an attraction with normal and express queues.
        :param name: Name of the attraction.
        :param ride_duration: Duration of the ride
        :param ride_capacity: Number of visitors that can be processed at once.
        """
        self.name = name
        self.regular_queue = deque()  
        self.fast_pass_queue = deque()
        self.ride_duration = ride_duration  # Time of each ride
        self.ride_capacity = ride_capacity # Max number of ppl a ride can take
        self.current_time = 10  # Simulation time in seconds
        self.total_served = 0  # cumulative number of served visitors 
        self.rides_served = 0 # Cumulative number of rides served in the current simulation, Flag for early termination
    
    def get_rides_served(self):
        return self.rides_served

    # def advance_time(self):
    #     # Update the current time based on the ride time
    #     self.current_time += self.ride_duration
    #     print(f"Time advanced to: {self.current_time} seconds.")

    def add_visitor(self, visitor):
        """
        :param visitor: a Visitor object
        """
        if isinstance(visitor, Visitor):
            if visitor.arrival_time <= self.current_time:
                if visitor.fast_pass == 1:
                    self.fast_pass_queue.append(visitor)
                    print(f"{visitor} added to the fast-pass queue at time {self.current_time}.") # for debugtest
                else:
                    self.regular_queue.append(visitor)
                    print(f"{visitor} added to the regular queue at time {self.current_time}.") # for debugtest
            else:
                print(f"{visitor} has not yet arrived at time {self.current_time}.")
        else:
            print("Only Visitor objects can be added to the queue.") # for debugtest

    def process_queue(self):
        # assumption: ride_capacity is EVEN number; fast-pass vs regular queue each take up 1/2 of ride_capacity
        # operation: take min(1/2*ride_capacity visitors out of fast_pass_queue, len(fast_pass_queue)), the rest from regular_queue
        
        riding_visitors = [] # list of visitors being served in the current ride
        
        # take min(1/2*ride_capacity visitors out of fast_pass_queue, len(fast_pass_queue))
        if self.fast_pass_queue: # while fast_pass_queue has visitors
            for _ in range(min(self.ride_capacity//2, len(self.fast_pass_queue))):
                person = self.fast_pass_queue.popleft()
                if person not in riding_visitors:
                    riding_visitors.append(person) # Get visitors in fast_pass_queue
                    print(f"{person} from Fast Pass Queue is being served.") # for debugging
                    self.total_served += 1
                    print("total serving so far:", self.total_served) # for debugging
        
        fast_pass_filled_ride = len(riding_visitors) # number of fast_pass visitors going to the current ride
        
        # the rest from regular_queue
        if self.regular_queue:
            for _ in range(min(self.ride_capacity - fast_pass_filled_ride, len(self.regular_queue))):
                person = self.regular_queue.popleft()
                if person not in riding_visitors:
                    riding_visitors.append(person) # Get visitors in regular_queue
                    print(f"{person} from Regular Queue is being served.") # for debugging
                    self.total_served += 1
                    print("total serving so far:", self.total_served) # for debugging
        
        if not self.fast_pass_queue and not self.regular_queue:
            print("No one is waiting in the queues at time", self.current_time)
        
        self.rides_served += 1
        print(f"Number of rides so far is{self.rides_served}") # debugging

        # The time now is right before the ride starts. We can update visitor's current time to calculate waiting time
        for visitor in riding_visitors:
            visitor.current_ride_start = self.current_time
            wait_time = visitor.current_ride_start - visitor.current_time # wait time = time the ride start - time visitor arrive at the queue
            print(f"{visitor} waited for {wait_time}") # debugging
            visitor.accumulated_waiting_time += wait_time
            # Then, update the time for Visitor objects 
            visitor.current_time += timedelta(minutes=self.ride_duration) 
        
        # Lastly, we update the time of this Attraction object:
        self.current_time += self.ride_duration
        print(f"The time at {self.name} now is {self.current_time}.")  

    def get_curr_crowd_level(self): # Note: should only be called AFTER calling process_queue()
        """
        :param time: a datetime object 
        """
        crowd_level = {'fast_pass_queue':len(self.fast_pass_queue), 
                       'regular_queue':len(self.regular_queue)}
        return self.current_time, crowd_level
    
    def get_curr_wait_time(self): # Note: should only be called AFTER calling process_queue()
        """
        :param time: a datetime object 
        """
        crowd_level = self.get_curr_crowd_level()[1]
        curr_wait_time = {'fast_pass_queue':math.floor(crowd_level['fast_pass_queue']/self.ride_capacity)*self.ride_duration, 
                          'regular_queue':math.floor(crowd_level['regular_queue']/self.ride_capacity)*self.ride_duration}
        return self.current_time, curr_wait_time
    

In [ ]:
class Utility: # Including toilets, dining outlets, souvenir shops
    def __init__(self, name: str, service_duration: int, util_capacity: int):
        """
        Initialize a utility in the part.
        :param name: Name of the utility.
        :param service_time: Duration spent inside the Utility
        :param util_capacity: Number of visitors that can be processed at once.
        """
        self.name = name
        self.service_duration = service_duration 
        self.util_capacity = util_capacity 
        self.queue = deque()
        self.serving_visitors = deque() # To track visitors being served at the current moment
        self.current_time = 10  # Simulation time in seconds
        self.total_served = 0  # cumulative number of served visitors 

    def add_visitor(self, visitor):
        """
        :param visitor: a Visitor object
        """
        if isinstance(visitor, Visitor):
            if visitor.arrival_time <= self.current_time:
                self.queue.append(visitor)
                print(f"{visitor} added to the queue at time {self.current_time}.") # for debugtest
            else:
                print(f"{visitor} has not yet arrived at time {self.current_time}.") # for debugtest
        else:
            print("Only Visitor objects can be added to the queue.") # for debugtest


    def process_queue(self):
        """
        Process the queue continuously, allowing visitors to enter as soon as space is available.
        """
        # Check if any serving visitors have completed their service duration and can leave
        self.update_serving_visitors()

        # Move visitors from the queue to service as capacity allows
        while len(self.serving_visitors) < self.util_capacity and self.queue:
            visitor = self.queue.popleft()
            visitor.service_start_time = self.current_time  # Set the start time for service
            visitor.service_end_time = self.current_time + timedelta(minutes=self.service_duration)  # Set end time
            visitor.waiting_time = (visitor.start_time - visitor.arrival_time).total_seconds() // 60  # In minutes
            self.serving_visitors.append(visitor)
            self.total_served += 1
            print(f"{visitor} is now being served. Waiting time was {visitor.waiting_time} minutes.")

    def update_serving_visitors(self):
        """
        Removes visitors from `serving_visitors` who have completed their service.
        """
        while self.serving_visitors and self.serving_visitors[0].service_end_time <= self.current_time:
            completed_visitor = self.serving_visitors.popleft()
            print(f"{completed_visitor} has completed their time in {self.name} and is leaving.")

    def advance_time(self, minutes: int):
        """
        Advances the current time by a specified number of minutes and processes the queue again.
        """
        self.current_time += timedelta(minutes=minutes)
        print(f"The time now is {self.current_time}.")
        self.process_queue()

    def get_curr_crowd_level(self):
        """
        Returns the current crowd level in the queue.
        """
        crowd_level = len(self.queue)
        print(f"At {self.current_time}, the crowd level is {crowd_level}")
        return self.current_time, crowd_level

    def get_curr_wait_time(self):
        """
        Returns the estimated wait time based on the current crowd level and service duration.
        """
        crowd_level = self.get_curr_crowd_level()[1]
        curr_wait_time = math.ceil(crowd_level / self.util_capacity) * self.service_duration
        print(f"At {self.current_time}, the estimated wait time is {curr_wait_time} minutes")
        return self.current_time, curr_wait_time


In [ ]:
class ThemePark:
    def __init__(self):

        self.global_time = None  # Overall time counter for the simulation, take from the time_slot object
        self.attractions = []
        self.visitors = []
   

    def add_attraction(self, attraction: Attraction):
        """
        Add an attraction to the park.
        :param attraction: Attraction object.
        """
        self.attractions.append(attraction)


    def output_waiting_times(self):
        return None


    def get_crowd_levels(self):
        return None



In [ ]:
# Create Visitor objects from csv file
# Columns needed: id, itinerary, fast_pass, arrival_time

visitor_generator_df = pd.read_csv('file path here')

visitors = [] # List to store Visitor objects
for index, row in visitor_generator_df.iterrows():
    new_visitor = Attraction(id=row['id'], itinerary=row['itinerary'], fast_pass=row['fast_pass'], arrival_time=row['arrival time']) # change according to the finalised csv
    visitors.append(new_visitor)

In [ ]:
# Create Attraction objects from csv file
# Columns needed: name: str, ride_duration: int, ride_capacity: int

attraction_generator_df = pd.read_csv('file path here')

attractions = {} # Dictionary to store Attraction objects
attractions_name = [] # List to store Attraction name, to link with Visitor action
for index, row in attraction_generator_df.iterrows():
    new_attraction = Attraction(name=row['name'], ride_duration=row['duration'], ride_capacity=row['capacity']) # change according to the finalised csv
    attractions[row['name']] = new_attraction # key = new_attraction_name, item = Attraction object new_attraction
    attractions_name.append(row['name'])



In [ ]:
# Example Usage
park = ThemePark(num_visitors=50)

# Create attractions with average wait times and capacity
roller_coaster = Attraction("Roller Coaster", avg_wait_time=45, capacity=20)
ferris_wheel = Attraction("Ferris Wheel", avg_wait_time=30, capacity=15)
haunted_house = Attraction("Haunted House", avg_wait_time=25, capacity=10)

# Add attractions to the park
park.add_attraction(roller_coaster)
park.add_attraction(ferris_wheel)
park.add_attraction(haunted_house)

# Assign visitors to queues (20% with express passes)
park.assign_visitors(express_pass_ratio=0.2)

# Run the simulation for a specified number of time steps
park.simulate(steps=60)

# Output total waiting times for each visitor
park.output_waiting_times()

# Output average crowd levels for each attraction
park.get_crowd_levels()

# Summary of average waiting times for analysis
waiting_times_summary = park.get_waiting_times_summary()
print("\nWaiting Times Summary:", waiting_times_summary)